### IMPORTACION DE LIBRERIAS

In [1]:
import pandas as pd
import ast
import matplotlib as plt
import numpy as np
import re
from datetime import datetime
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from rake_nltk import Rake

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rubio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### CARGA DEL DATASET

In [2]:
# Cargamos los datasets en formato csv y las convertimos en dataframes con pandas
df_movies = pd.read_csv(r'C:\Users\rubio\Documents\SoyHenry\Proyecto_individual_2\Dataset\movies_dataset.csv') # Dataset de peliculas
df_credits = pd.read_csv(r'C:\Users\rubio\Documents\SoyHenry\Proyecto_individual_2\Dataset\credits.csv') # Dataset del staff

C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\2661632247.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(r'C:\Users\rubio\Documents\SoyHenry\Proyecto_individual_2\Dataset\movies_dataset.csv') # Dataset de peliculas


### ETL (EXTRACCION, TRANSFORMACION Y CARGA)

In [3]:
# Eliminamos las columnas irrelevantes que no seran incluidas en el ETL ni el EDA
df_movies.drop(columns={'homepage', 'video', 'imdb_id', 'adult', 'original_title', 'poster_path'}, inplace=True)

In [4]:
# Hacemos un llenado de valores nulos o Nan con el valor numerico 0, para que en el futuro no nos moleste en el calculo de la columna 'return'
df_movies['revenue'].fillna(value=0, inplace=True)
df_movies['budget'].fillna(value=0, inplace=True)
print(df_movies['revenue'].isna().sum())  # Verificamos los valores faltantes en 'revenue'
print(df_movies['budget'].isna().sum())  # Verificamos los valores faltantes en 'budget'

0
0


C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\870035887.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_movies['revenue'].fillna(value=0, inplace=True)
C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\870035887.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [5]:
# Explotamos (en sentido figurado) y normalizamos la columna 'genres' debido a que tiene informacion anidada 
explode = df_movies['genres'].explode()
norm = pd.json_normalize(explode).reset_index(drop=True)

# Concatenamos la normalización con el dataframe original
df_movies = pd.concat([df_movies, norm], axis=1)

In [6]:
# Creamos una funcion para extraer valores de un diccionario dentro de una cadena 'str' ya que varias columnas tienen dicho formato
def explode_dict(value,key): # Asignamos las variables: 'value' seria el valor de la columna y 'key' la clave que queremos extrer
    if pd.isna(value):
        return np.nan
    try:
        dicc = ast.literal_eval(value)  # Convertimos el valor string en un diccionario
        if isinstance(dicc, dict):
            return dicc.get(key, np.nan)  # Extraemos únicamente el valor asociado a la clave 'key'
        else:
            return np.nan
    except (ValueError, SyntaxError):
        return np.nan

In [7]:
# Extraemos el nombre de la colección que está dentro de la columna 'belongs_to_collection' con la clave 'name'
df_movies['collection'] = df_movies['belongs_to_collection'].apply(explode_dict, key='name')
df_movies.drop(columns={'belongs_to_collection'}, inplace=True)  # Eliminamos la columna original una vez extraido el valor mencionado

In [8]:
# Creamos una funcion muy parecida a la anterior ya que hay columnas con una pequeña diferencia en los datos anidados con listas de diccionarios
def explode_list(value, key):
    if pd.isna(value):
        return np.nan
    try:
        lis = ast.literal_eval(value)
        if isinstance(lis, list): 
            names = [item.get(key) for item in lis if isinstance(item, dict)]
            return ', '.join(names) if names else np.nan  # Esto devuelve una cadena con los valores separados por comas
        else:
            return np.nan
    except (ValueError, SyntaxError):
        return np.nan

In [9]:
# Aplicamos la funcion para extraer los 'name' o nombres que hacen referencia al dato mas importante y luego se elimina la columna
df_movies['generos'] = df_movies['genres'].apply(explode_list, key='name')
df_movies.drop(columns={'genres'}, inplace=True)

df_movies['country'] = df_movies['production_countries'].apply(explode_list, key='name')
df_movies.drop(columns={'production_countries'}, inplace=True)

df_movies['company'] = df_movies['production_companies'].apply(explode_list, key='name')
df_movies.drop(columns={'production_companies'}, inplace=True)

df_movies['language'] = df_movies['spoken_languages'].apply(explode_list, key='name')
df_movies.drop(columns={'spoken_languages'}, inplace=True)

In [11]:
# Conviertimos la columna de presupuesto en tipo numerico para poder hacer calculos posteriormente, conviertiendo errores en campos nulos
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['revenue'].fillna(0)  # Hacemos llenado de valores faltantes en ganancia o 'revenue'
df_movies['budget'].fillna(0)  # Lo mismo con presupuesto o 'budget'

0        30000000.0
1        65000000.0
2               0.0
3        16000000.0
4               0.0
            ...    
45461           0.0
45462           0.0
45463           0.0
45464           0.0
45465           0.0
Name: budget, Length: 45466, dtype: float64

In [12]:
# Creamos la función para calcular el retorno que se calcula con los valores de 'budget' y 'revenue'
def calc_return(budget, revenue):
    try:
        b = int(budget)
        r = revenue
        if b == 0 or r == 0:
            return 0
        else:
            roi = r / b
            return roi
    except (ValueError, SyntaxError):
        return np.nan

In [13]:
# Aplicamos la función de calculo de retorno sobre las filas de las columnas mencionadas
df_movies['return'] = df_movies.apply(lambda row: calc_return(row['budget'], row['revenue']), axis=1)

In [14]:
# Eliminamos las filas donde la fecha de lanzamiento sea nula porque fue solicitado
df_movies.dropna(subset=['release_date'], inplace=True)

In [15]:
# Hacemos una función para cambiar el formato de las fechas
def format_date(column):
    try:
        date = datetime.strptime(column, '%Y-%m-%d')  # Convierte a objeto datetime
        return date
    except(ValueError, TypeError):
        return None

In [16]:
# Aplicamos la funcion para el formateo de fecha en la columna correspondiente
df_movies['release_date'] = df_movies['release_date'].apply(format_date)

In [17]:
# Creamos una funcion para extraer solo el año de las fechas de lanzamiento
def only_year(date):
    year = date.year
    return year

# Y aplicamos dicha funcion en la columna que corresponde
df_movies['release_year'] = df_movies['release_date'].apply(only_year)

In [18]:
# Convertimos la columna 'popularity' a tipo numérico para evitar conflictos mas adelante
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')

In [19]:
# Concatenamos los datasets de películas y créditos para unificar los datos y alinearlos
df = pd.concat([df_movies, df_credits], axis=1)

In [20]:
# Hacemos una función para extraer los actores de la columna 'cast' que estan anidados de manera similar a otras columnas anteriores
def extract_actors(columna):
    try:
        cast = ast.literal_eval(columna)
    except (ValueError, SyntaxError):
        return []
    
    return [actor.get('name') for actor in cast]

# Y aplicamos
df['actors'] = df['cast'].apply(extract_actors)

In [21]:
# Creamos una función para extraer los directores de la columna 'crew' que es mas compleja 
# ya que solo queremos el nombre del director entre otros miembros del staff
def extract_directors(crew_list):
    if isinstance(crew_list, str):
        crew_list = ast.literal_eval(crew_list)
    
    directores = [d['name'] for d in crew_list if d.get('job') == 'Director']
    return directores[0] if directores else None
    
# Y aplicamos
df['director'] = df['crew'].apply(extract_directors)

### ENDPOINT #1 cantidad_filmaciones_mes

In [22]:
# Generamos una nueva columna con el mes de lanzamiento 
df['release_month'] = df['release_date'].dt.month

In [23]:
# Hacemos un nuevo dataset solamente con las columnas que necesitaremos para el endpoint 1
df_endpoint1 = df[['title','release_month']]

In [24]:
# Remplazamos los valores del mes en numeros por el nombre del mes para luego resulte mas sencilla la funcion 
df_endpoint1['release_month'] = df_endpoint1['release_month'].replace({1:'enero',2:'febrero', 3:'marzo', 4:'abril', 5:'mayo', 6:'junio', 7:'julio', 8:'agosto', 9:'septiembre', 10:'octubre', 11:'noviembre', 12:'diciembre'})

C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\1750770614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_endpoint1['release_month'] = df_endpoint1['release_month'].replace({1:'enero',2:'febrero', 3:'marzo', 4:'abril', 5:'mayo', 6:'junio', 7:'julio', 8:'agosto', 9:'septiembre', 10:'octubre', 11:'noviembre', 12:'diciembre'})


In [25]:
# Hacemos la funcion que buscara todas las filmaciones que fueron lanzadas en el mes introducido 
def cantidad_filmaciones_mes(mes:str):
    filmaciones = df_endpoint1['title'][df_endpoint1['release_month'] == mes]
    cantidad = filmaciones.count()
    return f'{cantidad} películas fueron estrenadas en el mes de {mes}'

In [26]:
# Y finalmente probamos la funcion
cantidad_filmaciones_mes('enero')

'5912 películas fueron estrenadas en el mes de enero'

### ENDPOINT #2 cantidad_filmaciones_dia

In [27]:
# Generamos una nueva columna pero esta vez colocamos el dia en lugar del mes
df['release_day'] = df['release_date'].dt.day_name()

In [28]:
# Remplazamos los dias de la semana que estan en ingles por su equivalente en español
df['release_day'] = df['release_day'].replace({
    'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles',
    'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'
})

In [29]:
# Hacemos un dataframe orientado al uso del endpoint 2
df_endpoint2 = df[['title', 'release_day']]

In [30]:
# Creamos la funcion que consiste en devolver la cantidad de peliculas estrenadas en el dia ingresado
def cantidad_filmaciones_dia(dia:str):
    filmaciones = df_endpoint2['title'][df_endpoint2['release_day'] == dia]
    cantidad = filmaciones.count()
    return f'{cantidad} películas fueron estrenadas en el dia {dia}'

In [31]:
# Probamos dicha funcion
cantidad_filmaciones_dia('lunes')

'3503 películas fueron estrenadas en el dia lunes'

### ENDPOINT #3 score_titulo

In [32]:
# Generamos el dataframe para usar en el endpoint 3
df_endpoint3 = df[['title','popularity','release_year']]

In [33]:
# Generamos la funcion que busca la pelicula en el dataframe y recopila los datos solicitados
def score_titulo(titulo_de_la_filmacion:str):
    film = df_endpoint3[df_endpoint3['title'] == titulo_de_la_filmacion]
    title = film['title'].values[0]
    year = film['release_year'].values[0]
    score = film['popularity'].values[0]
    return f'La película {title} fue estrenada en el año {int(year)} con un score/popularidad de {score}'

In [34]:
# Probamos la funcion
score_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995 con un score/popularidad de 17.015539'

### ENDPOINT #4 votos_titulo

In [35]:
# Armamos el dataframe para este endpoint
df_endpoint4 = df[['title','release_year','vote_average','vote_count']]

In [36]:
# Hacemos una funcion parecida al endpoint 3 y recopila informacion sobre los votos y otras caracteristicas de la pelicula
def votos_titulo(titulo_de_la_filmacion:str):
    film = df_endpoint4[df_endpoint4['title'] == titulo_de_la_filmacion]
    valoraciones = film['vote_count'].values[0]
    promedio = film['vote_average'].values[0]
    año = film['release_year'].values[0]
    titulo = film['title'].values[0]
    if valoraciones < 2000:
        return 'no cumple con la cantidad de valoraciones minimas'
    else:
        return f'La película {titulo} fue estrenada en el año {int(año)}. La misma cuenta con un total de {valoraciones} valoraciones, con un promedio de {promedio}'


In [37]:
# Nos fijamos si funciona correctamente
votos_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995. La misma cuenta con un total de 2413.0 valoraciones, con un promedio de 6.9'

### ENDPOINT #5 get_actor

In [38]:
# Creamos el dataframe para su uso en el endpoint 5
df_endpoint5 = df[['title', 'actors', 'return']]

In [39]:
# Generamos la funcion donde se filtra las peliculas que contienen el nombre del actor en la lista de actores del df
def get_actor(nombre:str):
    peliculas = df_endpoint5[df_endpoint5['actors'].apply(lambda actors: nombre in actors)] # Aca el filtro de peliculas con el actor mencionado
    pelis = peliculas['title'].tolist()
    retorno = peliculas['return'].tolist()
    cantidad = len(pelis)
    ganancia = sum(retorno)
    promedio = ganancia/cantidad
    return print(f'El actor {nombre} ha participado en {cantidad} filmaciones, el mismo ha conseguido un retorno de {round(ganancia,3)} millones con un promedio de {round(promedio,3)} millones por filmación')
    

In [40]:
# Probamos la funcion creada
get_actor('Adam Sandler')

El actor Adam Sandler ha participado en 47 filmaciones, el mismo ha conseguido un retorno de 72.031 millones con un promedio de 1.533 millones por filmación


### ENDPOINT #6 get_director

In [41]:
# Ultimo dataframe para su uso en el endpoint 6
df_endpoint6 = df[['title', 'return','director','release_date','budget','revenue']]

In [42]:
# Esta funcion es mas sencilla que get_actor debido a que en la columna directores solo hay un solo nombre
def get_director(nombre:str):
    peliculas_director = df_endpoint6[df_endpoint6['director'] == nombre]
    if peliculas_director.empty:
        return f"No se encontraron películas para el director: {nombre}"
    retorno_total = peliculas_director['return'].sum()
    informacion_peliculas = peliculas_director[['title', 'release_date', 'return', 'budget', 'revenue']]
    return {
        'nombre_director': nombre,
        'retorno_total': retorno_total,
        'informacion_peliculas': informacion_peliculas
    }

In [43]:
# Finalmente probamos la funcion
get_director('Christopher Nolan')

{'nombre_director': 'Christopher Nolan',
 'retorno_total': 35.13215293098237,
 'informacion_peliculas':                                         title release_date    return  \
 2466                                Following   1998-09-12  8.080333   
 4099                                  Memento   2000-10-11  4.413677   
 5254                                 Insomnia   2002-05-24  2.472062   
 10122                           Batman Begins   2005-06-10  2.494791   
 11354                            The Prestige   2006-10-19  2.741908   
 12481                         The Dark Knight   2008-07-16  5.430046   
 15480                               Inception   2010-07-14  5.159580   
 18252                   The Dark Knight Rises   2012-07-16  4.339756   
 22878  The Inevitable Defeat of Mister & Pete   2013-10-11  0.000000   
 25896                              Stuntwoman   1977-10-05  0.000000   
 25974            To Grandmother's House We Go   1992-12-06  0.000000   
 44688               

### SISTEMA DE RECOMENDACION 

In [44]:
# Armamos el dataframe con las columnas necesarias para hacer un sistema de recomendacion con NearestNeighbors
df_reco = df[['title','overview','generos']]

# Reemplazamos valores nulos en 'title', 'overview' y 'generos' con una cadena vacía para que no haya conflictos a la hora de procesar y extraer info
df_reco['title'] = df_reco['title'].fillna('')
df_reco['overview'] = df_reco['overview'].fillna('')
df_reco['generos'] = df_reco['generos'].fillna('')

# Inicializamos el extractor RAKE para insertarla en la siguiente funcion
rake = Rake()

# Creamos la funcion para extraer palabras clave que vamos a aplicar a una de las columnas
def extract_keywords(text):
    rake.extract_keywords_from_text(text)
    return ' '.join(rake.get_ranked_phrases())

# Aplicamos la funcion de extracción de palabras clave a la columna 'overview'
df_reco['keywords'] = df_reco['overview'].apply(extract_keywords)

# Creamos una nueva columna combinando 'title', 'generos' y 'keywords' para tener toda la info necesaria para una buen sistema de recomendacion
df_reco['combined_text'] = df_reco['generos'] + ' ' + df_reco['keywords']

# Aplicamos una vectorizacion con TfidfVectorizer sobre el texto combinado anteriormente
vect = TfidfVectorizer(stop_words='english', max_features=5000)  # Podemos ajustar 'max_features' para limitar la cantidad de palabras
vect_matrix = vect.fit_transform(df_reco['combined_text'])

# Usamos NearestNeighbors con los parametros elegidos y entrenamos el modelo
nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
nn_model.fit(vect_matrix)

# Hacemos la función de recomendación
def recomendacion(titulo):
    if titulo not in df_reco['title'].values:
        return f"No se encontró la película: {titulo}"

    idx = df_reco[df_reco['title'] == titulo].index[0] # Aca se obtiene el indice de la pelicula ingresada

    distances, indices = nn_model.kneighbors(vect_matrix[idx], n_neighbors=6) # El modelo busca y devuelve los indices de las peliculas similares

    similar_titles = df_reco['title'].iloc[indices[0][1:]]  # Excluimos nuestra pelicula y devolvemos hasta 5 resultados
    
    return similar_titles.tolist()

C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\2072408389.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reco['title'] = df_reco['title'].fillna('')
C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\2072408389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reco['overview'] = df_reco['overview'].fillna('')
C:\Users\rubio\AppData\Local\Temp\ipykernel_15652\2072408389.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [45]:
# Probamos el sitema de recomendacion y evaluamos el resultado
recomendacion('Toy Story')

['Toy Story 3',
 'The 40 Year Old Virgin',
 'The Champ',
 'Andy Kaufman Plays Carnegie Hall',
 "Andy Hardy's Blonde Trouble"]

### Guardamos los archivos para el EDA y el posterior deployment

In [47]:
df.to_csv('df_eda')

In [46]:
# Guardamos los df de cada endpoint y del sistema de recomendacion en formato parquet
df_endpoint1.to_parquet('end1.parquet')
df_endpoint2.to_parquet('end2.parquet')
df_endpoint3.to_parquet('end3.parquet')
df_endpoint4.to_parquet('end4.parquet')
df_endpoint5.to_parquet('end5.parquet')
df_endpoint6.to_parquet('end6.parquet')
df_reco.to_parquet('reco.parquet')